# Copyright

Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)


In [8]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# random seed a könnyebb reprodukálhatósághoz
seed = 77
np.random.seed(seed)

## Sentiment elemzés (tetszik/nem tetszik) szóbeágyazás alapján filmkritikákból

In [9]:
# Betöltjük az adatokat (Keras támogatott)
top_words=6000 # Limitáljuk a szavak számát, 6000 leggyakoribbat hagyjuk meg, többit kidobjuk
skip_top=0 # Abból is kihagyhatjuk mondjuk a 20 leggyakoribbat -> érdemes így is futtatni és összehasonlítani az eredményt

(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words,skip_top=skip_top)

# Nincs validációs halmaz, lecsípünk a teszthalmazból validációhoz
X_val=X_test[:15000]
X_test=X_test[15000:]
y_val=y_test[:15000]
y_test=y_test[15000:]

# Adatok alakja, osztályok száma
print("Adataink: ")
print(X_train.shape)
print(y_train.shape)

print(y_val.shape)
print(y_test.shape)

print("Osztályok: ")
print(np.unique(y_train))

print("Szavak száma: ")
print(len(np.unique(np.hstack(X_train))))

print("Egy minta: ")
print(X_val[0])
print(y_val[0])

Adataink: 
(25000,)
(25000,)
(15000,)
(10000,)
Osztályok: 
[0 1]
Szavak száma: 
5998
Egy minta: 
[1, 89, 27, 2, 2, 17, 199, 132, 5, 4191, 16, 1339, 24, 8, 760, 4, 1385, 7, 4, 22, 1368, 2, 16, 5149, 17, 1635, 7, 2, 1368, 9, 4, 1357, 8, 14, 991, 13, 877, 38, 19, 27, 239, 13, 100, 235, 61, 483, 2, 4, 7, 4, 20, 131, 1102, 72, 8, 14, 251, 27, 1146, 7, 308, 16, 735, 1517, 17, 29, 144, 28, 77, 2305, 18, 12]
1


## Padding egyenlő hosszra

In [4]:
# Ennyi szót fogunk kritikánként figyelembe venni, ha ennél rövidebb kritika van, azt nullákkal kipaddingeli a pad_sequences().
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print("Első minta padding után")
print(X_train[0])

Első minta padding után
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

## Modell összeállítása, tanítás és tesztelés

In [5]:
embedding_dim=50

model = Sequential()
# Mi csináljuk az embeddinget is
model.add(Embedding(top_words, embedding_dim, input_length=max_words))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 50)       300000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 25000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 250)           6250250     flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             251         dense_1[0][0]                    
Total params: 6550501
_____________________________________________________________________

In [6]:
# Tanítás (csak 2 epoch, nincs túltanulás elleni védelmünk)
model.fit(X_train, y_train, validation_data=(X_val, y_val), nb_epoch=2, batch_size=128, verbose=1)

Train on 25000 samples, validate on 15000 samples
Epoch 1/2
25000/25000 [==============================] - 27s - loss: 0.4540 - acc: 0.7652 - val_loss: 0.2866 - val_acc: 0.8777
Epoch 2/2
25000/25000 [==============================] - 27s - loss: 0.1495 - acc: 0.9460 - val_loss: 0.3136 - val_acc: 0.8735
Accuracy: 86.62%


In [ ]:
# Kiértékelés a tesztadatokon
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))